## https://www.kernix.com/blog/an-efficient-recommender-system-based-on-graph-database_p9

In [17]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import json
from tornado import ioloop, httpclient
import pickle

In [2]:
# Loading user-related data
users = pd.read_csv('data/ml-100k/u.user', sep='|', header=None, names=['id','age','gender','occupation','zip code'], encoding='cp1252')
n_u = users.shape[0]

# Loading genres of movies
genres = pd.read_csv('data/ml-100k/u.genre', sep='|', header=None, names=['name', 'id'], encoding='cp1252')
n_g = genres.shape[0]

# Loading item-related data
# Format : id | title | release date | | IMDb url | "genres"
# where "genres" is a vector of size n_g : genres[i]=1 if the movie belongs to genre i
movie_col = ['id', 'title','release date', 'useless', 'IMDb url']
movie_col = movie_col + genres['id'].tolist()
movies = pd.read_csv('data/ml-100k/u.item', sep='|', header=None, names=movie_col, encoding='cp1252')
movies = movies.fillna('unknown')
n_m = movies.shape[0]

# Loading ratings
rating_col = ['user_id', 'item_id','rating', 'timestamp']
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t' ,header=None, names=rating_col, encoding='cp1252')
n_r = ratings.shape[0]

In [3]:
from imdb import IMDb
ia = IMDb()
from lxml import html
import requests
#
movies['synopsis'] = ''
movies['storyline'] = ''
movies['description']= ''
movies['text'] = ''

In [4]:
movies.head()

,id,title,release date,useless,IMDb url,0,1,2,3,4,...,13,14,15,16,17,18,synopsis,storyline,description,text
0,1,Toy Story (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,,,,
1,2,GoldenEye (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,1,0,0,,,,
2,3,Four Rooms (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,1,0,0,,,,
3,4,Get Shorty (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,,,,
4,5,Copycat (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,1,0,0,,,,


In [ ]:
url = 'http://www.imdb.com/find?q={}&s=tt&exact=true&ref_=fn_tt_ex'.format('Toy Story (1995)')
page = requests.get(url)
tree = html.fromstring(page.content)
# Get the movie id 
idm = re.match('tt(\d+)',tree.xpath('//td[@class="result_text"]//a/@href')[0].split('/')[2]).group(1)
print (idm)
# # Toy%20Story%20%281995%29

In [ ]:
ia.get_movie(idm)

In [ ]:
s_result = ia.search_movie('Toy Story')

# Print the long imdb canonical title and movieID of the results.
for item in s_result:
   print item['long imdb canonical title'], item.movieID

In [ ]:
movies['synopsis'] = ''
movies['storyline'] = ''
movies['description']= ''
movies['text'] = ''

In [9]:
from imdb import IMDb
ia = IMDb()
from lxml import html
import requests
#

# Looping over movies
for index, row in movies.iterrows():
    if movies.loc[index,'storyline'] == '':
        if (row['IMDb url'] != 'unknown'):
            # Get the html content 
            try:
                url = 'http://www.imdb.com/find?q={}&s=tt&exact=true&ref_=fn_tt_ex'.format(row['title'].encode('utf-8'))
                page = requests.get(url)
                tree = html.fromstring(page.content)
                # Get the movie id 
                idm = re.match('tt(\d+)',tree.xpath('//td[@class="result_text"]//a/@href')[0].split('/')[2]).group(1)
    
                #             page = requests.get(row['IMDb url'])
                #             tree = html.fromstring(page.content)
                # Get the movie id 
                #             idm = tree.xpath('/html/head/meta[6]')[0].get('content')[2:]
                print (idm)
                
                # Get the story line
                page = requests.get('http://www.imdb.com/title/tt{}/?ref_=fn_tt_tt_1'.format(idm))
                tree = html.fromstring(page.content)
                movies.loc[index, 'storyline'] = tree.xpath('//div[@id="titleStoryLine"]/div[1]/p')[0].text
                # Get the movie using the IMDB library based on the movie id
    #             m = ia.get_movie(idm)
                # Get the descreption 
    #             movies.loc[index, 'description'] = m.get('plot outline')
                # Get the synopsis
    #             ia.update(m, 'synopsis')
    #             if m.get('synopsis') is not None:
    #                 movies.loc[index, 'synopsis'] = m.get('synopsis')
            except:
                print row['IMDb url']
    #             pass
    # Create a text field to concatenate all existing summaries
    movies.loc[index, 'text'] = movies.loc[index, 'description'] + ' ' + movies.loc[index, 'storyline'] + ' '+ movies.loc[index, 'synopsis']


http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)
http://us.imdb.com/M/title-exact?Se7en%20(1995)
http://us.imdb.com/M/title-exact?Usual%20Suspects,%20The%20(1995)
http://us.imdb.com/M/title-exact?Postino,%20Il%20(1994)
http://us.imdb.com/M/title-exact?Gazon%20maudit%20(1995)
http://us.imdb.com/M/title-exact?Badkonake%20Sefid%20(1995)
http://us.imdb.com/M/title-exact?Birdcage,%20The%20(1996)
http://us.imdb.com/M/title-exact?Brothers%20McMullen,%20The%20(1995)
http://us.imdb.com/M/title-exact?Doom%20Generation,%20The%20(1995)
http://us.imdb.com/M/title-exact?Net,%20The%20(1995)
http://us.imdb.com/M/title-exact?To%20Wong%20Foo,%20Thanks%20for%20Everything!%20Julie%20Newmar%20(1995)
http://us.imdb.com/M/title-exact?Dolores%20Claiborne%20(1994)
6855750
http://us.imdb.com/M/title-exact?Star%20Wars%20(1977)
http://us.imdb.com/M/title-exact?Madness%20of%20King%20George,%20The%20(1994)
http://us.imdb.com/Title?L%E9on+(1994)
http://us.imdb.com/M/title-exact?Santa%20Clause,%20The%20(1

http://us.imdb.com/Title?Romeo+%2B+Juliet+(1996)
http://us.imdb.com/M/title-exact?Transformers:%20The%20Movie,%20The%20(1986)
http://us.imdb.com/M/title-exact?Day%20the%20Earth%20Stood%20Still,%20The%20(1951)
http://us.imdb.com/M/title-exact?Heathers%20(1989)
http://us.imdb.com/M/title-exact?American%20Werewolf%20in%20London,%20An%20(1981)
http://us.imdb.com/M/title-exact?Amityville%201992:%20It's%20About%20Time%20(1992)
http://us.imdb.com/M/title-exact?Amityville%20Horror,%20The%20(1979)
http://us.imdb.com/M/title-exact?Amityville%20Curse,%20The%20(1990)
http://us.imdb.com/M/title-exact?Birds,%20The%20(1963)
http://us.imdb.com/M/title-exact?Blob,%20The%20(1958)
http://us.imdb.com/M/title-exact?Body%20Snatcher,%20The%20(1945)
http://us.imdb.com/M/title-exact?Omen,%20The%20(1976)
http://us.imdb.com/M/title-exact?Police%20Story%204:%20First%20Strike%20(1996)
http://us.imdb.com/M/title-exact?Crossing%20Guard,%20The%20(1995)
http://us.imdb.com/M/title-exact?Como%20agua%20para%20chocolate%2

http://us.imdb.com/M/title-exact?Swimming%20with%20Sharks%20(1995)
http://us.imdb.com/M/title-exact?Baby-Sitters%20Club,%20The%20(1995)
http://us.imdb.com/M/title-exact?Richie%20Rich%20(1994)
http://us.imdb.com/M/title-exact?Boys%20Life%20(1995)
http://us.imdb.com/M/title-exact?In%20the%20Mouth%20of%20Madness%20(1995)
http://us.imdb.com/M/title-exact?Air%20Up%20There,%20The%20(1994)
http://us.imdb.com/M/title-exact?Program,%20The%20(1993)
http://us.imdb.com/M/title-exact?Shadow,%20The%20(1994)
http://us.imdb.com/M/title-exact?Thirty-Two%20Short%20Films%20About%20Glenn%20Gould%20(1993)
http://us.imdb.com/M/title-exact?Celluloid%20Closet,%20The%20(1995)
http://us.imdb.com/M/title-exact?Great%20Day%20in%20Harlem,%20A%20(1994)
http://us.imdb.com/M/title-exact?Great%20White%20Hype,%20The%20(1996)
http://us.imdb.com/M/title-exact?Arrival,%20The%20(1996)
http://us.imdb.com/M/title-exact?Phantom,%20The%20(1996)
http://us.imdb.com/M/title-exact?Power%2098%20(1995)
http://us.imdb.com/M/title-exa

http://us.imdb.com/M/title-exact?Boys%20of%20St.%20Vincent,%20The%20(1993)
http://us.imdb.com/M/title-exact?Pred%20dozhdot%20(1994)
http://us.imdb.com/M/title-exact?Fresa%20y%20chocolate%20(1993)
http://us.imdb.com/M/title-exact?Nuits%20fauves,%20Les%20(1992)
http://us.imdb.com/M/title-exact?Family%20Thing,%20A%20(1996)
http://us.imdb.com/M/title-exact?Dellamorte%20Dellamore%20(1994)
http://us.imdb.com/M/title-exact?Bewegte%20Mann,%20Der%20(1994)
http://us.imdb.com/M/title-exact?Secret%20Agent,%20The%20(1996)
http://us.imdb.com/M/title-exact?Ai%20no%20Corrida%20(1976)
http://us.imdb.com/M/title-exact?Goofy%20Movie,%20A%20(1995)
http://us.imdb.com/M/title-exact?Scout,%20The%20(1994)
http://us.imdb.com/M/title-exact?Night%20Falls%20on%20Manhattan%20(1997)
http://us.imdb.com/M/title-exact?Awfully%20Big%20Adventure,%20An%20(1995)
http://us.imdb.com/M/title-exact?Poison%20Ivy%20II%20(1995)
http://us.imdb.com/Title?Pr%EAt-%E0-Porter+(1994)
http://us.imdb.com/Title?N%E9nette+et+Boni+(1996)
ht

http://us.imdb.com/M/title-exact?Apur%20Sansar%20(1959)
http://us.imdb.com/M/title-exact?Wedding%20Gift,%20The%20(1994)%20(TV)
http://us.imdb.com/M/title-exact?Fear,%20The%20(1995)
http://us.imdb.com/M/title-exact?Good%20Man%20in%20Africa,%20A%20(1994)
http://us.imdb.com/Title?Object+of+My+Affection,+The+(1998)
http://us.imdb.com/M/title-exact?De%20Eso%20No%20Se%20Habla%20(1993)
http://us.imdb.com/M/title-exact?Enfer,%20L'%20(1994)
http://us.imdb.com/M/title-exact?Aiqing%20Wansui%20(1994)
http://us.imdb.com/M/title-exact?Being%20Human%20(1993)
http://us.imdb.com/M/title-exact?Amazing%20Panda%20Adventure,%20The%20(1995)
http://us.imdb.com/M/title-exact?Beans%20of%20Egypt,%20Maine,%20The%20(1994)
http://us.imdb.com/M/title-exact?Scarlet%20Letter,%20The%20(1926)
http://us.imdb.com/M/title-exact?Cienie%20(1988)
http://us.imdb.com/M/title-exact?Show,%20The%20(1995)
http://us.imdb.com/M/title-exact?Glass%20Shield,%20The%20(1994)
http://us.imdb.com/M/title-exact?Hunted,%20The%20(1995)
http://

In [22]:
with open('data/movies.pickle','wb') as f:
    pickle.dump(movies,f)

In [15]:
movies_storyline = movies[movies['storyline'] != '']

In [16]:
movies_storyline.head()

,id,title,release date,useless,IMDb url,0,1,2,3,4,...,13,14,15,16,17,18,synopsis,storyline,description,text
0,1,Toy Story (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,,\nA little boy named Andy loves to be in his r...,,\nA little boy named Andy loves to be in his ...
1,2,GoldenEye (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,1,0,0,,\nWhen a deadly satellite weapon system falls ...,,\nWhen a deadly satellite weapon system falls...
2,3,Four Rooms (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,1,0,0,,\nThis movie features the collaborative direct...,,\nThis movie features the collaborative direc...
3,4,Get Shorty (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,,"\nSome guys get all the luck, whether they lik...",,"\nSome guys get all the luck, whether they li..."
4,5,Copycat (1995),01-Jan-1995,unknown,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,1,0,0,,"\nIn San Francisco, the criminal psychologist ...",,"\nIn San Francisco, the criminal psychologist..."


In [ ]:

from imdb import IMDb
ia = IMDb()
from lxml import html
import requests
#
movies['synopsis'] = ''
movies['storyline'] = ''
movies['description']= ''
movies['text'] = ''

http_client = httpclient.AsyncHTTPClient()

def populate_storyline(page,index):
    print 'gogo'
    tree = html.fromstring(page.content)
    movies.loc[index, 'storyline'] = tree.xpath('//div[@id="titleStoryLine"]/div[1]/p')[0].text
    movies.loc[index, 'text'] = movies.loc[index, 'description'] + ' ' + movies.loc[index, 'storyline'] + ' '+ movies.loc[index, 'synopsis']
    

def populate_idm(page,index):
    tree = html.fromstring(page.content)
    idm = re.match('tt(\d+)',tree.xpath('//td[@class="result_text"]//a/@href')[0].split('/')[2]).group(1)
    url = 'http://www.imdb.com/title/tt{}/?ref_=fn_tt_tt_1'.format(idm)
    cb_pop_story = lambda page: populate_storyline(page,index)
    http_client.fetch(url.strip(), cb_pop_story, method='GET')
    
    


# Looping over movies
for index, row in movies.iterrows():
    if index > 1:
        break
    if (row['IMDb url'] != 'unknown'):
        # Get the html content 
        try:
            url = 'http://www.imdb.com/find?q={}&s=tt&exact=true&ref_=fn_tt_ex'.format(row['title'])
            cb_pop_idm = lambda page: populate_idm(page,index)
            http_client.fetch(url.strip(), cb_pop_idm, method='GET')
        except IndexError:
            print row['IMDb url']
#             pass
    # Create a text field to concatenate all existing summaries
    
ioloop.IOLoop.instance().start()

In [ ]:
movies.head()

In [ ]:
idm = 0114709
page = requests.get('http://www.imdb.com/title/tt{}/?ref_=fn_tt_tt_1'.format(idm))
tree = html.fromstring(page.content)
print tree.xpath('//div[@id="titleStoryLine"]/div[1]/p')[0].text

In [ ]:
from nltk.tokenize import RegexpTokenizer
rt = RegexpTokenizer(r'\w+')
s = 'this is a; text-with tokens, and with punctuations.'


s.apply((lambda x: ' '.join(rt.tokenize(x))))